<a href="https://colab.research.google.com/github/Hyrysake/My_Data_Science/blob/main/Hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import pandas as pd
import time
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import IsolationForest

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
main_path = "/content/drive/My Drive/Colab Notebooks/data"

path_to_idle = pd.DataFrame()
path_to_running = pd.DataFrame()
path_to_stairs = pd.DataFrame()
path_to_walking = pd.DataFrame()

for i in os.listdir(main_path):
    i_path = os.path.join(main_path, i)
    if os.path.isdir(i_path):
        for file in os.listdir(i_path):
            end_path = os.path.join(i_path, file)

            df = pd.read_csv(end_path)
            df['activity_folder'] = i

            if i == 'idle':
                path_to_idle = pd.concat([path_to_idle, df], ignore_index=True)
            elif i == 'running':
                path_to_running = pd.concat([path_to_running, df], ignore_index=True)
            elif i == 'stairs':
                path_to_stairs = pd.concat([path_to_stairs, df], ignore_index=True)
            elif i == 'walking':
                path_to_walking = pd.concat([path_to_walking, df], ignore_index=True)

data = pd.concat([path_to_idle,
                  path_to_running,
                  path_to_stairs,
                  path_to_walking], ignore_index=True)

Нова таблиця, для зручного працювання


In [11]:
path_to_csv = 'indicators.csv'
data.to_csv(path_to_csv, index=False)

result_data = pd.read_csv('indicators.csv')
result_data

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity_folder
0,0.430956,0.009577,9.773131,idle
1,0.416591,-0.038307,9.753978,idle
2,0.435745,-0.033519,9.749189,idle
3,0.411803,0.028730,9.739613,idle
4,0.421380,-0.090980,9.730036,idle
...,...,...,...,...
193855,-0.837971,-11.458650,1.000776,walking
193856,-1.637634,-5.497087,-5.702988,walking
193857,-0.679953,-8.202534,1.039084,walking
193858,1.886631,-6.909666,0.842759,walking


In [12]:
X = result_data.drop('activity_folder', axis=1)
X.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,0.430956,0.009577,9.773131
1,0.416591,-0.038307,9.753978
2,0.435745,-0.033519,9.749189
3,0.411803,0.028730,9.739613
4,0.421380,-0.090980,9.730036


In [13]:
y = result_data['activity_folder']
y.head()

0    idle
1    idle
2    idle
3    idle
4    idle
Name: activity_folder, dtype: object

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Середні значення наших данних

In [15]:
X_train['average over time'] = X_train.mean(axis=1)
X_train.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,average over time
57654,8.609550,17.899052,-1.733402,8.258400
12457,0.248997,-0.023942,9.777920,3.334325
177098,-1.134852,-7.776367,-0.134075,-3.015098
191995,-0.153229,-10.046070,-2.633622,-4.277640
118812,10.817003,-6.588843,-1.403002,0.941719


In [16]:
X_test['average over time'] = X_test.mean(axis=1)
X_test.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,average over time
171248,-1.125275,-8.461109,-1.058237,-3.548207
58417,5.094861,0.814029,-2.288857,1.206678
54186,5.607220,3.308787,-9.581595,-0.221863
131711,-1.843535,10.381259,-1.388637,2.383029
27960,0.220267,-0.033519,9.773131,3.319960


Моделі машинного навчання

Випадковий ліс

In [17]:
random_forest_model = RandomForestClassifier(class_weight='balanced')

start_time = time.time()
random_forest_model.fit(X_train, y_train)
end_time = time.time()

training_time_seconds = end_time - start_time
training_time_minutes, training_time_seconds = divmod(training_time_seconds, 60)
print("Model training time:", int(training_time_minutes), "minutes", int(training_time_seconds), "seconds")

Model training time: 0 minutes 21 seconds


Опорні вектори

In [18]:
support_vector_method = SVC()

start_time = time.time()
support_vector_method.fit(X_train, y_train)
end_time = time.time()

training_time_seconds = end_time - start_time
training_time_minutes, training_time_seconds = divmod(training_time_seconds, 60)
print("Model training time:", int(training_time_minutes), "minutes", int(training_time_seconds), "seconds")

Model training time: 6 minutes 28 seconds


Прогоноз даних

In [19]:
start_time = time.time()
random_forest_model_prediction = random_forest_model.predict(X_test)
end_time = time.time()

prediction_time_seconds = end_time - start_time
prediction_time_minutes, prediction_time_seconds = divmod(prediction_time_seconds, 60)
print("Model training time:", int(prediction_time_minutes), "minutes", int(prediction_time_seconds), "seconds")

Model training time: 0 minutes 0 seconds


In [20]:
start_time = time.time()
support_vector_method_prediction = support_vector_method.predict(X_test)
end_time = time.time()

prediction_time_seconds = end_time - start_time
prediction_time_minutes, prediction_time_seconds = divmod(prediction_time_seconds, 60)
print("Model training time:", int(prediction_time_minutes), "minutes", int(prediction_time_seconds), "seconds")

Model training time: 1 minutes 45 seconds


Розраховуємо точність

In [21]:
random_forest_model_prediction_precision = accuracy_score(y_test, random_forest_model_prediction)
print("accuracy for the vicinity forest model")
print(random_forest_model_prediction_precision)

random_forest_model_prediction_summary = classification_report(y_test, random_forest_model_prediction)
print("classification report for the vicinity forest model")
print(random_forest_model_prediction_summary)

accuracy for the vicinity forest model
0.9996904982977406
classification report for the vicinity forest model
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      7786
     running       1.00      1.00      1.00     25542
      stairs       1.00      0.99      1.00      1290
     walking       1.00      1.00      1.00     13847

    accuracy                           1.00     48465
   macro avg       1.00      1.00      1.00     48465
weighted avg       1.00      1.00      1.00     48465



In [22]:
support_vector_method_prediction_precision = accuracy_score(y_test, support_vector_method_prediction)
print("accuracy for the SVM model")
print(support_vector_method_prediction_precision)

support_vector_method_prediction_summary = classification_report(y_test, support_vector_method_prediction)
print("classification report for the SVM model")
print(support_vector_method_prediction_summary)

accuracy for the SVM model
0.8927473434437223
classification report for the SVM model
              precision    recall  f1-score   support

        idle       0.96      0.99      0.97      7786
     running       0.93      0.90      0.92     25542
      stairs       1.00      0.00      0.01      1290
     walking       0.80      0.91      0.85     13847

    accuracy                           0.89     48465
   macro avg       0.92      0.70      0.69     48465
weighted avg       0.90      0.89      0.88     48465



**Висновок:**
Метод випадкового лісу кращи практично у всьому. В швидкості та точності.
Якщо проаналізуємо звіти по 2-м методам, то:
у моделі випадкового лісу значення precision та f1-score мають високий показник = 1. Це означає, що модель дуже гарно передбачає усі класи, не допускає помилок і практично не пропускає об'єкти ні одного класу (значення recall).
у моделі опорних векторів гірші результати по звіту, параметр precision показує що точність не ідеальна, і передбачення будуть з помилками. Требі звернути увагу на клас stairs - recall = 0, отже ні один з прикладів stairs не був коректно класифікований моделлю.